# 1 - Introduction
## Contexte

Dans le cadre de mon arrivée en CDI à [SQLI](https://www.sqli.com/int-en) et en vue de la préparation du contrat CIFRE, je me suis posé comme défie de résoudre une compétition sur Kaggle, j'ai souhaité utiliser mes connaissances acquises précédemment pour la résolution de ce dataset [Mushrooms images classification 215](https://www.kaggle.com/datasets/daniilonishchenko/mushrooms-images-classification-215/data).

## Présenation du dataset
> **Un dataset (jeu de données)** : Ensemble de données (ici des images) organisée en classe.

Le jeu de données est composé de 3 122 images dont les images sont 512 pixels de largeur et de longueur. Ces images sont réparties en 215 classes de champignons différents.

## Objectifs

L'objectif principal est de confronter plusieurs algorithmes de classification d'image dans plusieurs conditions différentes.

Le second est de pouvoirs, faire un état de l'art de mes connaissances dans le domaine de la classification d'image.

# 2 - Préparation des données (data wrangling)
## Définitions

> **Préparation des données (ou [data wrangling](https://datascientest.com/data-wrangling-tout-savoir))** :
> *Qu'est-ce donc ?* C'est un processus précédant toutes tâches d'exploitation des données telles que des processus d'analyse ou machine learning.
> *Comment ?* Il n'existe pas de méthodes types déjà écrites, la méthode est à concevoir selon les données.
> *Pourquoi ?* Le processus permet de transformer des données récupérées brutes, souvent non structurées, en données structurées et exploitables facilement.

> **Augmentation des données (ou [data augmentation](https://datascientest.com/data-augmentation-tout-savoir))** :
> *Qu'est-ce donc ?* C’est un processus qui créée de la donnée en se fondant sur des données déjà existantes.
> *Comment ?* Les nouvelles données sont créées en apportant des modifications sur certaines caractéristiques. (ex : pour les images, on joue alors sur l’échelle, l’orientation, les couleurs, la luminosité, etc).
> *Pourquoi ?* Dans certains cas, pour qu’un algorithme arrive à généraliser le modèle, dans le cas où les données en entrée ne sont pas variées et/ou peu nombreuse.

> **Nuancier de gris (greyscale) :** Une palette de couleur composée uniquement de niveaux de gris. Sur une image, cela signifie que chaque pixel correspond à une quantité de lumière, blanc étant l'intensité la plus forte et noir la plus faible.

> **Nuancier de couleur (RGB (Red, Green, Blue)) :** Une palette de couleur composée de trois canaux : rouge, vert et bleu. Chaque canal défini l'intensité de la couleur. Ces trois canaux sont ensuite *superposés* pour former une image colorée.

> **Train set :** est un sous-ensemble d'un dataset, utilisé en entrée d'un algorithme pour ajuster les paramètres d'un modèle.

> **Validation set :** est le sous-ensemble d'un dataset, utilisé pour valider l'ajustement d'un modèle à la fin d'une époque (définition dans la partie 4).

> **Test set :** est le sous-ensemble d'un dataset, utilisé pour valider l'ajustement final d'un modèle à la fin de la phase d'entraînement.

Dans le processus défini, nous devons générer quatre datasets : avec et sans data augmentation puis en nuancier de gris et de couleurs.

In [ ]:
import os
import shutil
from zipfile import ZipFile

from PIL import Image
from PIL import ImageOps

import numpy as np
from skimage import filters
from skimage import util
import random

import tensorflow as tf
import pandas as pd

In [ ]:
path_root = f"./dataset"

path_datas_zip = f"{path_root}/archive.zip"
path_data_root = f"{path_root}/data/"
path_data_directory = f"{path_data_root}/data/data/"

path_datasets_corrected = f"{path_root}/data-corrected/"

path_datasets = f"{path_root}/Fruits-100/"

path_datasets_augmented = f"{path_root}/data-augmented-corrected/"
path_datasets_augmented_trainTest = f"{path_root}/dataset-augmented/"

In [ ]:
def removeAndCreateFolder(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path, 0o777)

def trainAndValidateSet(path, ratio):
    return (
        tf.keras.preprocessing.image_dataset_from_directory(
            path,
            validation_split = ratio,
            subset = "training",
            seed = 42,
            labels = "inferred"
        ),
        tf.keras.preprocessing.image_dataset_from_directory(
            path,
            validation_split = ratio,
            subset = "validation",
            seed = 42,
            labels = "inferred"
        )
    )

def CopyDataset(path_src, path_dst):
    removeAndCreateFolder(path_datasets)
    removeAndCreateFolder(path_dst)
    
    removeAndCreateFolder(f"{path_dst}/test/")
    removeAndCreateFolder(f"{path_dst}/temp/")
    temp_set , test_set = trainAndValidateSet(path_src, 0.2)
    for class_name in temp_set.class_names:
        os.mkdir(f"{path_dst}/temp/{class_name}")
    for file_path in temp_set.file_paths:
        shutil.copyfile(file_path, f"{path_dst}/temp/{file_path.replace(path_src, '')}")


    for class_name in test_set.class_names:
        os.mkdir(f"{path_dst}/test/{class_name}")
    for file_path in test_set.file_paths:
        shutil.copyfile(file_path, f"{path_dst}/test/{file_path.replace(path_src, '')}")


    removeAndCreateFolder(f"{path_dst}/train/")
    removeAndCreateFolder(f"{path_dst}/validation/")
    train_set , validation_set = trainAndValidateSet(f"{path_dst}/temp/", 0.25)    
    for class_name in train_set.class_names:
        os.mkdir(f"{path_dst}/train/{class_name}")
    for file_path in train_set.file_paths:
        shutil.copyfile(file_path, file_path.replace('/temp/', '/train/'))


    for class_name in validation_set.class_names:
        os.mkdir(f"{path_dst}/validation/{class_name}")
    for file_path in validation_set.file_paths:
        shutil.copyfile(file_path, file_path.replace('/temp/', '/validation/'))
        
    shutil.rmtree(f"{path_dst}/temp/")

## Extraction du zip

In [ ]:
removeAndCreateFolder(path_data_root)

pictures = pd.DataFrame(columns = ["FileName", "Width", "Height", "Mode", "Label"])

if len(os.listdir(path_data_root)) == 0:
    with ZipFile(path_datas_zip, 'r') as zip_file:
        zip_file.extractall(path_data_root)
        print(f"The '{path_datas_zip}' folder has been extracted.")

for folder in os.listdir(path_data_directory):
    for file in os.listdir(f"{path_data_directory}/{folder}"):
        image = Image.open(f"{path_data_directory}/{folder}/{file}")

        pictures = pd.concat([
            pictures,
            pd.DataFrame({
                "FileName": [file],
                "Width": [float(image.size[0])],
                "Height": [float(image.size[1])],
                "Mode": [image.mode],
                "Label": [folder],
            })
        ])
            
    print(f"The '{folder}' folder has been read.")

## Corection des données

In [ ]:
removeAndCreateFolder(path_datasets_corrected)

if len(os.listdir(path_datasets_corrected)) == 0:
    for index, row in pictures.iterrows():
        image = Image.open(f"{path_data_directory}/{row['Label']}/{row['FileName']}")

        if not os.path.exists(f"{path_datasets_corrected}/{row['Label']}"):
            os.mkdir(f"{path_datasets_corrected}/{row['Label']}", 0o777)
        
        image.copy().convert("RGB").save(f"{path_datasets_corrected}/{row['Label']}/{row['FileName']}", "PNG")

## Préparation *sans* data augmentation

In [ ]:
if not os.path.exists(path_datasets):
    CopyDataset(path_datasets_corrected, path_datasets)

## Préparation *avec* data augmentation

Pour la data augmentation, j'ai décidé, à partir des images du dataset, de générer six images de plus pour chaque image.
Le processus choisira aléatoirement des valeurs pour les quatre paramètres, selon les conditions suivantes :
- Rotation : $[0 ; 360]$,
- Miroire : $\{True, False\}$,
- Filtre Gaussien : $[0 ; 1]$
- Bruitage : $[0 ; 0.05]$

In [ ]:
removeAndCreateFolder(path_datasets_augmented)

if len(os.listdir(path_datasets_augmented)) == 0:
    for index, row in pictures.iterrows():
        image = Image.open(f"{path_data_directory}/{row['Label']}/{row['FileName']}")

        if not os.path.exists(f"{path_datasets_augmented}/{row['Label']}"):
            os.mkdir(f"{path_datasets_augmented}/{row['Label']}", 0o777)

        image.copy().convert("RGB").save(f"{path_datasets_augmented}/{row['Label']}/0-{row['FileName']}", "PNG")

        for i in range(1, 15):
            imageTmp = image.copy()
            imageTmp = np.array(imageTmp)
            
            imageTmp = filters.gaussian(imageTmp, sigma = random.uniform(0, 1))
            imageTmp = util.random_noise(imageTmp, mode = "speckle", var = random.uniform(0, 0.05))
            imageTmp = (imageTmp * 255).astype(np.uint8)

            imageTmp = Image.fromarray(imageTmp)
            imageTmp = imageTmp.rotate(random.randint(0, 360))

            if random.choice([True, False]):
                imageTmp = ImageOps.mirror(imageTmp)

            imageTmp = imageTmp.convert("RGB")
            
            imageTmp.save(f"{path_datasets_augmented}/{row['Label']}/{i}-{row['FileName']}", "PNG")

In [ ]:
if not os.path.exists(path_datasets_augmented_trainTest):
    CopyDataset(path_datasets_augmented, path_datasets_augmented_trainTest)

## Fonction d'importation des datasets

In [ ]:
def loadDataSet(path_dataset, shape):
    return (
        tf.keras.preprocessing.image_dataset_from_directory(
            f"{path_dataset}/train/",
            validation_split = 0.2,
            subset = "training",
            seed = 42,
            labels = "inferred",
            image_size = shape
        ),
        tf.keras.preprocessing.image_dataset_from_directory(
            f"{path_dataset}/train/",
            validation_split = 0.2,
            subset = "validation",
            seed = 42,
            labels = "inferred",
            image_size = shape
        ),
        tf.keras.preprocessing.image_dataset_from_directory(
            f"{path_dataset}/test/",
            labels = "inferred",
            image_size = shape
        )
    )

# 3 - Présentation des modèles
## Type de problème

> **Apprentissage supervisé (Supervised Learning) :** les algorithmes utilisent des datasets labellisé/étiqueté qui est utilisé pour entraîner l'algorithme. L'algorithme peut ensuite faire des prédictions sur des données inédites.

> **Apprentissage non supervisé (Unsupervised Learning) :** ces algorithmes sont entrainés avec uniquement les données en entrée, il n'y a donc pas de labels/étiquettes.

> **Apprentissage par renforcement (Reinforcement Learning) :** ces algorithmes « forment » un système dans un environnement dynamique, qui lui fournie en retour des « bon point » et « mauvais point ». Le but pour le système est d'obtenir le maximum de bon point.

![Schéma des applications du machine learning regroupé par type d'apprentissage.](./images/MachineLearning-Introduction%20-%20Type%20d'apprentissage.drawio.png "Schéma des applications du machine learning regroupé par type d'apprentissage.")

Le type de problème que nous rencontrons est une classification d'images en 215 classes, ce type de problème est un apprentissage supervisé.

Dans les problèmes de classification d'image, nous utilisons des réseaux de neurones, ce type d'algorithme font partie de la famille du deep learning, lui-même faisant partie de la famille du machine learning.

![Comparaison deep learning, machine learning et intelligence artificielle](./images/MachineLearning-Introduction%20-%20Familles.drawio.png "Comparaison deep learning, machine learning et intelligence artificielle")

In [ ]:
from tensorflow.keras import layers, Model, losses, activations, applications

## Perceptron multi-couches
### Définitions

Le Perceptron est une modélisation informatique du neurone formel apparue en 1957 par Frank Rosenblatt avec les règles d'apprentissage du Perceptron. L'algorithme se décline en deux implémentations : mono-couche et multicouche.

#### **Neurone formel (neurone McCulloch-Pitts)**

Avant de commencer l'explication d'un neurone formel, un petit rappel du fonctionnement d'un neurone biologique s'impose. Le neurone est divisé en deux parties : le corps cellulaire et l'axone.
- Le corps cellulaire est composé de plusieurs dendrites pour simplifier l'entrée du neurone connectée au noyau enveloppé dans une membrane cytoplasmique et qui peut être ici décrit comme un agent décideur.
- L'axone peut être simplifié comme la sortie du neurone.

<img alt="Schéma simplifié d&amp;amp;amp;amp;amp;#39;un neurone schoolmov" height="400" src="https://images.schoolmouv.fr/gssm-svt-img30p.png" title="Schéma simplifié d&amp;amp;amp;amp;amp;#39;un neurone schoolmov" width="650"/>

Effectué par Warren McCulloch et Walter Pitts en 1943, le neurone formel est la formalisation mathématique du neurone biologie et fonctionne de la même manière, nous avons des "dendrites" qui sont la somme des valeurs d'entrée (représenté par un vecteur noté $\overrightarrow{X}$) multiplié par leurs poids respectifs. Cette somme est ensuite soumise au noyau, ce dernier va "traiter" la valeur avec une fonction mathématique que l'on appelle fonction d'activation. À la sortie de la fonction, nous retrouvons la valeur calculé fait par le neurone que nous pouvons identifier comme l'axone sur le neurone biologique.

![Formalisation mathématique du neurone biologique](./images/MachineLearning-MLP%20-%20Neurone%20Formel.drawio.png "Formalisation mathématique du neurone biologique")

> Légende :
> - $x_0$ le biais,
> - $\overrightarrow{X}$ le vecteur d'entrée,
> - $f$ fonction d'activation souvent sigmoïd,
> - $\theta$ le seuil de la fonction d'activation, pas obligatoire dépend de la fonction choisie,
> - $Y$ la sortie du neurone.

#### **Vecteur d'entrée** $\overrightarrow{X}$

C'est le paramètre envoyé en entrée d'un réseau de neurone, les paramètres sont noté mathématiquement : $\overrightarrow{X} \begin{pmatrix}
x_1 \\
x_{...} \\
x_d \\
\end{pmatrix}$.

#### **Fonction d'activation**

Une fonction reprend le principe du potentiel d'activation d'un neurone biologique, c'est-à-dire une fonction qui définit le seuil pour la sortie du neurone.

 **Exemple - la fonction sigmoïde (fonction logistique)** : $ f(x) = \frac{1}{1 + \mathrm{e}^{-x}} $ où $ f(x) \in ] 0, 1 [ $.

![Courbe de la fonction sigmoïde](./images/MachineLearning-MLP%20-%20Fonction%20sigmoïde.drawio.png "Courbe de la fonction sigmoïde")

**Exemple  - la fonction linéaire** : $ f(x) = x $
![Couche de la fonction linéaire](./images/MachineLearning-MPL%20-%20Fonction%20linéaire.drawio.png "Couche de la fonction linéaire")

#### **Perceptron simple**

Le perceptron simple est un classificateur binaire basé sur le neurone formel. C'est en 1957, que Frank Rosenblatt proposa dans ses travaux *Perceptron Learning Rule*, des règles décrivant les règles d'apprentissage du perceptron, l'algorithme agence le coefficient des poids lié au vecteur d'entrée le plus optimal possible.

#### **Perceptron multicouche**

Le perceptron multicouche a été proposé par Paul Werbos en 1974 et mis au point par David Rumelhart en 1986, le perceptron multicouche reprend les travaux du perceptron en y ajoutant l'algorithme de descente de gradient.

#### **Dropout :**

C'est une méthode utilisée pour mettre aléatoirement le vecteur d'entrée à 0, ce qui permet aux unités de ne pas trop se spécialiser.

### Schéma

![Image perceptron](./images/MachineLearning-MLP%20-%20Schéma.drawio.png "Image perceptron")

### Modèle

In [ ]:
def perceptron(shape, rate_dropout = 0.):
    input = layers.Input(shape = shape, name = "input")

    fully_connected1 = layers.Flatten(input_shape = shape, name = "fullyConnected1")(input)
    fully_connected2 = layers.Dense(512, activation = "sigmoid", name = "fullyConnected2")(fully_connected1)
    fully_connected3_1 = layers.Dense(512, activation = "sigmoid", name = "fullyConnected3-1")(fully_connected2)
    fully_connected3_2= layers.Dropout(rate_dropout, name = "fullyConnected3-2")(fully_connected3_1)

    output = layers.Dense(215, name = "output")(fully_connected3_2)

    return Model(inputs = input, outputs = output, name = "Perceptron")

## Convolutional neural network (CNN)
### Définitions

Les CNN sont des réseaux de neurones, qui comportent deux parties :
- La première partie est du traitement d'image, cette partie va encoder l'image en faisant ressortir des paternes/caractéristiques de l'image dans un vecteur de sorti. L'image est compressée, mais le nombre de dimensions du vecteur en sorti est supérieur à l'image en entrée.
- La seconde partie est le classificateur, souvent un pecerptron multicouche, le but de cette couche est de combiner les paternes/caractéristiques extraites pour les classer.

![CNN](./images/MachineLearning-CNN%20-%20schema.drawio.png "Schéma du fonctionnement d'un CNN")

#### Couches Conv

La couche de convolution est la couche appliquant un [filtre de convolution](http://mathinfo.alwaysdata.net/2016/11/filtres-de-convolution/) aux pixels d'une image. Elle est souvent représentée comme une fenêtre coulissante, de gauche à droite et de haut en bas, ici la fenêtre étant le filtre.

Il est à noter qu'un filtre de convolution s'applique aussi bien à une image qu'un signal temporel.

La représentation mathématique d'un filtrage par convolution pour une intensité est la suivante : $$ O(x;y) = \sum_{i = 0}^{N} \left( \sum_{j = 0}^{M} K(i;j) \times I(x-1+i;y-1+i) \right) $$

> Légende :
> $O(x;y)$ : la fonciton qui calcule la convolution d'une position $x$ et $y$ sur une image en nuancier de gris,
> $N$ et $M$ : la largeur et la hauteur du filtre de convolution,
> $K(i;j)$ : la fonction de renvoie de la valeur du filtre à la position $i$ et $j$,
> $I(x-1+i;y-1+i)$ : la fonction de renvoie de l'intentité sur l'image initial.

Sur cette couche nous avons retrouvons quatres paramètres :
- la taille du filtre : notée $N$ et $M$,
- la taille du stride : notée $S$, c'est le nombre de pas de déplacement de la fenêtre a chaque itération sur une image).
- La taille de sortie du filtre.
- Le "padding" : "same" signifie qu'un cadre de pixel égale à zéro est ajouté sur les bordures et valide" signifie qu'aucune modification n'est apportée.

![Image de convolution](./images/MachineLearning-CNN%20-%20convolution.drawio.png "Image de convolution")

#### Couches ReLu

La couche ReLu (Rectified Linear Units) est souvent interprété comme une couche d'activation, de la même manière que pour le neurone formel. Dans de nombreux frameworks, la couche ReLu n'existe pas et est directement implémenté dans la couche Conv.

> *Exemples de fonction :*
> - Remplacer les valeurs négatives par des zéros : $ f(x) = \left\{\begin{array}{ll}0 \to x \le  0 \\ x \to x \gt 0 \end{array} \right. $,
![Courbe de la fonction Relu](./images/MachineLearning-CNN%20-%20ReLu.drawio.png "Courbe de la fonction Relu")
> - Fonction de normalisation : $ f(x) = \frac{x - x_{min}}{x_{max} - x_{min}} $ .

#### Couches Pool

La couche de Pooling est très souvent placé en sortie de couche Conv après une correction avec la couche ReLu, elle a pour but de réduire la taille de l'image tout en préservant les paternes/caractéristiques essentielles.

Il existe plusieurs fonctions de Pooling, la plus utilisée étant le max-pooling.

Au même titre que la couche de Conv, il faut imaginer une fenêtre coulissante, contrairement à la couche Conv, nous ne gardons pas la somme du produit entre l'image et le filtre de convolution, mais uniquement la valeur la plus grande.

![Image de max-pooling](./images/MachineLearning-CNN%20-%20maxPooling.drawio.png "Image max-pooling")

#### Couches Fully Connected

La couche Fully Connected est un perceptron multicouche.

### AlexNet
#### Présentation

AlexNet est une architecture de Deep learning conçue par Alex Krizhevsky et Ilya Sutskever à l'université de Toronto publié dans [ImageNet Classification with Deep Convolutional Neural Networks](https://proceedings.neurips.cc/paper_files/paper/2012/hash/c399862d3b9d6b76c8436e924a68c45b-Abstract.html). L'architecture a remporté l'épreuve ImageNet en 2012.

> ImageNet est une base données d'image annotée a destination de travaux de recherche sur la vision par ordinateur.

#### Schéma

!["Schema AlexNet"](./images/MachineLearning-CNN%20-%20AlexNet.drawio.png "Schema AlexNet")

#### Modèle

In [ ]:
def alex_net(shape):
    input = layers.Input(shape = shape, name = "input")

    # Couches de traitement d'image
    conv1_1 = layers.Conv2D(filters = 96, kernel_size = 11, strides = 4, activation = "relu", name = "conv1-1")(input)
    conv1_2 = layers.MaxPool2D(pool_size = 3, strides = 2, name = "conv1-2")(conv1_1)

    conv2_1 = layers.Conv2D(filters = 256, kernel_size = 5, padding = "same", activation = "relu", name = "conv2-1")(conv1_2)
    conv2_2 = layers.MaxPool2D(pool_size = 3, strides = 2, name = "conv2-2")(conv2_1)

    conv3_1 = layers.Conv2D(filters = 384, kernel_size = 3, padding = "same", activation = "relu", name = "conv3-1")(conv2_2)
    conv3_2 = layers.Conv2D(filters = 384, kernel_size = 3, padding = "same", activation = "relu", name = "conv3-2")(conv3_1)
    conv3_3 = layers.Conv2D(filters = 256, kernel_size = 3, padding = "same", activation = "relu", name = "conv3-3")(conv3_2)
    conv3_4 = layers.MaxPool2D(pool_size = 3, strides = 2, name = "conv3-4")(conv3_3)

    # Couches de classification
    fully_connected1 = layers.Flatten(name = "fullyConnected1")(conv3_4)

    fully_connected2_1 = layers.Dense(4096, activation = "relu", name = "fullyConnected2-1")(fully_connected1)
    fully_connected2_2 = layers.Dropout(0.5, name = "fullyConnected2-2")(fully_connected2_1)

    fully_connected1_1 = layers.Dense(4096, activation = "relu", name = "fullyConnected3-1")(fully_connected2_2)
    fully_connected1_2 = layers.Dropout(0.5, name = "fullyConnected3-2")(fully_connected1_1)

    output = layers.Dense(255, name = "output")(fully_connected1_2)

    return Model(inputs = input, outputs = output, name = "AlexNet")

### Visual Geometry Group (VGG)
#### Présentation

[VGG](https://doi.org/10.48550/arXiv.1409.1556) est un réseau de neurones convolutionnels conçue par K. Simonyan et A. Zisserman à l'université d'Oxord et qui a gagné la compétition ILSVRC (ImageNet Large Scale Visual Recognition Challenge) en 2014.

#### Définition

Fonction d'activation softmax : Elle converti, un vecteur de $ K $ nombré réel en une distribution de probabilités. La fonciton est notée $  \sigma(z)_{j} = \frac{\mathrm{e}^{\mathcal{z}_{j}}}{\sum_{k=1}^{K} \mathrm{e}^{\mathcal{z}_{k}}} $ avec $ j \in {1, ..., K} $.

#### Schema

VGG16 c'est 13 couches de convolution et 3 couches en fully-connected.
VGG19 c'est 16 couches de convolution et 3 couches en fully-connected.

![Schémas comparatifs de VGG 16 et 19](./images/MachineLearning-CNN%20-%20VGG.drawio.png "Schémas comparatifs de VGG 16 et 19")

#### Modèle

In [ ]:
def vgg_block(x, depth, number_convolution, number_block):
    for i in range(1, number_convolution + 1):
        x = layers.Conv2D(filters = depth, kernel_size = 3, padding = "same", activation = "relu", name = f"VGG{number_block}-{i}")(x)

    return layers.MaxPool2D(pool_size = 2, strides = 2, name = f"VGG{number_block}-{number_convolution + 2}")(x)

##### VGG16

In [ ]:
def vgg_16(shape):
    input = layers.Input(shape = shape, name = "input")

    # Couches de traitement d'image
    vgg1 = vgg_block(input, 64, 2, 1)
    vgg2 = vgg_block(vgg1, 128, 2, 2)
    vgg3 = vgg_block(vgg2, 256, 3, 3)
    vgg4 = vgg_block(vgg3, 512, 3, 4)
    vgg5 = vgg_block(vgg4, 512, 3, 5)

    # Couches de classification
    fully_connected1 = layers.Flatten(name = "fullyConnected1")(vgg5)
    fully_connected2 = layers.Dense(4096, activation = "relu", name = "fullyConnected2")(fully_connected1)
    fully_connected3 = layers.Dense(4096, activation = "relu", name = "fullyConnected3")(fully_connected2)
    output = layers.Dense(255, activation = "softmax", name = "output")(fully_connected3)

    return Model(inputs = input, outputs = output, name = "VGG16")

##### VGG19

In [ ]:
def vgg_19(shape):
    input = layers.Input(shape = shape, name = "input")

    # Couches de traitement d'image
    vgg1 = vgg_block(input, 64, 2, 1)
    vgg2 = vgg_block(vgg1, 128, 2, 2)
    vgg3 = vgg_block(vgg2, 256, 4, 3)
    vgg4 = vgg_block(vgg3, 512, 4, 4)
    vgg5 = vgg_block(vgg4, 512, 4, 5)

    # Couches de classification
    fully_connected1 = layers.Flatten(name = "fullyConnected1")(vgg5)
    fully_connected2 = layers.Dense(4096, activation = "relu", name = "fullyConnected2")(fully_connected1)
    fully_connected3 = layers.Dense(4096, activation = "relu", name = "fullyConnected3")(fully_connected2)
    output = layers.Dense(255, activation = "softmax", name = "output")(fully_connected3)

    return Model(inputs = input, outputs = output, name = "VGG19")

### Residual Network (ResNet)
#### Présentation

La classe des réseaux de neurones avec résidue ont été indroduit dans l'article [Deep Residual Learning for Image Recognition](https://doi.org/10.48550/arXiv.1512.03385) par Kaiming He, Xiangyu Zhang et al. tous chercheur chez Microsoft en décembre 2015. Le réseau de neurone a été évaluer sur ImageNet de 2012.

#### Block résiduel

Dans un réseau de neurone avec résidue, il faut imaginer plusieurs sous réseau empiler les une après les autres.

![Schéma d'un résiduel bloc](./images/MachineLearning-CNN%20-%20Residual%20block.drawio.png "Schéma d'un résiduel bloc")

Chaque sous-réseau est appelé bloc résiduel, chaque bloc utilise la technique des connexions à saut, cela signifie que l'entrée du bloc est additionné avec la sortie du bloc. La fonction résiduel est notée de la manière suivante : $F(x) := H(x) - x \Leftrightarrow H(x) := F(x) + x$, avec $x$ l'entrée du bloc et $F(x)$ la sortie calculée du bloc et $H(x)$ la sortie du bloc avec résidus.

Lorsque les dimensions du vecteur d'entrée et le vecteur calculé par le bloc ne coïncide pas, il existe plusieurs méthodes :
- Nous appliquons une couche de convolution au vecteur d'entrée afin de réduire les dimensions du vecteur.
- Ne pas appliquer l'addition entre les vecteurs.

#### Schéma

![Schéma comparatif de ResNet 34 et 50](./images/MachineLearning-CNN%20-%20ResNet.drawio.png "Schéma comparatif de ResNet 34 et 50")

#### Modèle

In [ ]:
def conv_block(x, name, f, k, s = 1, with_relu = True):
    m = layers.Conv2D(filters = f, kernel_size = k, strides = s, padding = "same", name = f"{name}-1")(x)
    m = layers.BatchNormalization(name = f"{name}-2")(m)

    if with_relu:
        m = layers.ReLU(name = f"{name}-3")(m)

    return m

def residual_block(x, number_block, configs, repetition, is_residual = True):
    name = f"{'residual' if is_residual else 'plain'}{number_block}"

    for i in range(1, repetition+1):
        n = 0
        x_skip = x
        for config in configs:
            n += 1

            x = conv_block(
                x = x,
                name = f"{name}-{i}-{n}",
                f = config["f"],
                k = config["k"],
                s = 2 if i == 1 and n == 1 and number_block > 1 else 1,
                with_relu = len(configs) != n or is_residual == False
            )

        if is_residual and x.shape == x_skip.shape:
            n += 1
            x = layers.Add(name = f"{name}-{i}-{n}")([x_skip, x])

        x = layers.ReLU(name = f"{name}-{i}-{n + 1}")(x)

    return x

#### Avec 34 couches

In [ ]:
def resnet_34_layers(shape, is_residual = True):
    input = layers.Input(shape = shape, name = "input")
    name = f"{'residual' if is_residual else 'plain'}"

    # Couches de traitement d'image
    conv1_1 = layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, padding = "same", name = f"conv1-1")(input)
    conv1_2 = layers.MaxPool2D(pool_size = 3, strides = 2, padding = "same", name = f"conv1-2")(conv1_1)

    residual_block1 = residual_block(conv1_2, 1, [
        { "k": 3, "f": 64 },
        { "k": 3, "f": 64 }
    ], 3 , is_residual)

    residual_block2 = residual_block(residual_block1, 2, [
        { "k": 3, "f": 128 },
        { "k": 3, "f": 128 }
    ], 4, is_residual)

    residual_block3 = residual_block(residual_block2, 3, [
        { "k": 3, "f": 256 },
        { "k": 3, "f": 256 }
    ], 6, is_residual)

    residual_block4 = residual_block(residual_block3, 4, [
        { "k": 3, "f": 512 },
        { "k": 3, "f": 512 }
    ], 3, is_residual)

    conv2 = layers.GlobalAvgPool2D(name = "conv2")(residual_block4)

    # Couches de classification
    output = layers.Dense(215, activation = "softmax")(conv2)

    return Model(inputs = input, outputs = output, name = "ResNet-34")

#### Avec 50 couches

In [ ]:
def resnet_50_layers(shape, is_residual = True):
    input = layers.Input(shape = shape, name = "input")

    # Couches de traitement d'image
    conv1_1 = layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, padding = "same", name = f"conv-1-1")(input)
    conv1_2 = layers.MaxPool2D(pool_size = 3, strides = 2, padding = "same", name = f"conv-1-2")(conv1_1)

    residual_block1 = residual_block(conv1_2, 1, [
        { "k": 1, "f": 64, "s": 1 },
        { "k": 3, "f": 64, "s": 1 },
        { "k": 1, "f": 4*64, "s": 1 }
    ], 3, is_residual)

    residual_block2 = residual_block(residual_block1, 2, [
        { "k": 1, "f": 128, "s": 2 },
        { "k": 3, "f": 128, "s": 1 },
        { "k": 1, "f": 4*128, "s": 1 }
    ], 4, is_residual)

    residual_block3 = residual_block(residual_block2, 3, [
        { "k": 1, "f": 256, "s": 2 },
        { "k": 3, "f": 256, "s": 1 },
        { "k": 1, "f": 4*256, "s": 1 }
    ], 6, is_residual)

    residual_block4 = residual_block(residual_block3, 4, [
        { "k": 1, "f": 512, "s": 2 },
        { "k": 3, "f": 512, "s": 1 },
        { "k": 1, "f": 4*512, "s": 1 }
    ], 3, is_residual)

    conv2 = layers.GlobalAvgPool2D(name = "conv2")(residual_block4)

    # Couches de classification
    output = layers.Dense(215, activation = "softmax")(conv2)

    return Model(inputs = input, outputs = output, name = "ResNet-34")

### MobilNet
#### Présentation

La classe de réseau de neurone MobilNet a été publier en 2017 dans l'article [MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://doi.org/10.48550/arXiv.1704.04861 "MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications") par Andrew G. Howard, Menglong Zhu et al.

La deuxième version a été publier en 2019 dans l'article [MobileNetV2: Inverted Residuals and Linear Bottlenecks](https://doi.org/10.48550/arXiv.1801.04381 "MobileNetV2: Inverted Residuals and Linear Bottlenecks") par Mark Sandler, Andrew Howard et al.

Cette classe de réseau a été créée pour répondre au besoin d'utiliser la vision par ordinateur sur mobiles et autre appareille embarquées n'ayant pas la même puissance qu'un PC classique.

#### Convolution Depthwise

La convolution classique applique un filtre de convolution sur tous les canaux d'entrée en même temps.

La convolution deepthwise applique un filtre sur chaque canal d'entrée séparé, la logique apliqué est la suivante :
- Un tenseur d'entrée à 3 dimensions est divisé en canaux distincts.
- Pour chaque canal, l'entrée est convoluée avec un filtre (2D).
- La sortie de chaque canal est ensuite empilée pour obtenir la sortie sur l'ensemble du tenseur 3D.

![Schéma comparatif entre la convolution et la convolution depthwise](./images/MachineLearning-CNN%20-%20Depthwise.drawio.png "Schéma comparatif entre la convolution et la convolution depthwise")

#### ReLu6

La fonction relue reprend la fonction ReLu précédement décrite mais la valeur maximum est à 6.

Toutes les valeurs négatives sont remplacé par zéros et toutes les valeurs au dessus de six sont remplacé par six : $ f(x) = \left\{\begin{array}{ll} 0 \to x \le  0 \\ x \to 0 > x < 6 \\ 6 \to x \geq 6 \end{array} \right. $.

![Courbe de la fonction Relu plafonnée à 6](./images/MachineLearning-CNN%20-%20ReLu%206.drawio.png "Courbe de la fonction Relu plafonnée à 6")

#### Linear BottleNeck

Le goulot d'étranglement linéaire est dans le cas d'un réseau MobileNet V2 un bloc résiduel inversé. Cela signifie que les dimensions des fenêtres sont : large → étroit → large.

Le bloc résiduel classique est le suivant : étroit → large → étroit.

![Schéma d'un goulot d'étranglement pour  MobileNet V2](./images/MachineLearning-CNN%20-%20Linear%20Bottlenecks.drawio.png "Schéma d'un goulot d'étranglement pour  MobileNet V2")

#### Modèle
##### V1

![Schéma du MobileNet V1](./images/MachineLearning-CNN%20-%20MobileNet%20V1.drawio.png "Schéma du MobileNet V1")

In [ ]:
def depth_wise_block(x, f, s, number_block):
    x = layers.DepthwiseConv2D(kernel_size = 3, strides = s, padding = 'same', activation = "relu", name = f"depthWise{number_block}-1")(x)
    return layers.Conv2D(filters = f, kernel_size = 1, padding = 'same', activation = "relu", name = f"depthWise{number_block}-2")(x)

In [ ]:
def mobilenet_v1(shape):
    input = layers.Input(shape = shape, name = "input")

    # Couches de traitement d'image
    conv1 = layers.Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same', activation = "relu", name = "conv1")(input)

    depthWise1 = depth_wise_block(conv1, 64, 1, 1)

    depthWise2 = depth_wise_block(depthWise1, 128, 2, 2)
    depthWise3 = depth_wise_block(depthWise2, 128, 1, 3)

    depthWise4 = depth_wise_block(depthWise3, 256, 2, 4)
    depthWise5 = depth_wise_block(depthWise4, 256, 1, 5)

    depthWise6 = depth_wise_block(depthWise5, 512, 2, 6)
    depthWise7 = depth_wise_block(depthWise6, 512, 1, 7)
    depthWise8 = depth_wise_block(depthWise7, 512, 1, 8)
    depthWise9 = depth_wise_block(depthWise8, 512, 1, 9)
    depthWise10 = depth_wise_block(depthWise9, 512, 1, 10)
    depthWise11 = depth_wise_block(depthWise10, 512, 1, 11)

    depthWise12 = depth_wise_block(depthWise11, 1024, 2, 12)
    depthWise13 = depth_wise_block(depthWise12, 1024, 1, 13)

    # Couches de classification
    fullyConnected1 = layers.Flatten(name = "fullyConnected1")(depthWise13)
    fullyConnected2 = layers.Dense(1024, activation = "relu", name = f"fullyConnected2")(fullyConnected1)
    output = layers.Dense(255, activation = "softmax", name = "output")(fullyConnected2)

    return Model(inputs = input, outputs = output, name = "MobileNetV1")

##### v2

![Schéma du MobilNet V2](./images/MachineLearning-CNN%20-%20MobileNet%20V2.drawio.png "Schéma du MobilNet V2")

In [ ]:
def bottleneck(x, t, c, s, block_number):
    m = layers.Conv2D(filters = x.shape[-1] * t, kernel_size = 1, padding = "same", name = f"bottleneck{block_number}-1")(x)
    m = layers.BatchNormalization(name = f"bottleneck{block_number}-2")(m)
    m = layers.ReLU(max_value = 6., name = f"bottleneck{block_number}-3")(m)

    m = layers.DepthwiseConv2D(kernel_size = 3, strides = s, padding = "same", name = f"bottleneck{block_number}-4")(m)
    m = layers.BatchNormalization(name = f"bottleneck{block_number}-5")(m)
    m = layers.ReLU(max_value = 6., name = f"bottleneck{block_number}-6")(m)

    m = layers.Conv2D(filters = c, kernel_size = 1, padding = "same", name = f"bottleneck{block_number}-7")(m)
    m = layers.BatchNormalization(name = f"bottleneck{block_number}-8")(m)

    if s == 1 and x.shape == m.shape:
        m = layers.Add(name = f"bottleneck{block_number}-9")([m, x])

    return m

In [ ]:
def mobilenet_v2(shape):
    input = layers.Input(shape = shape, name = "inputLayers")

    # Couches de traitement d'image
    conv1_1 = layers.Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = "same", name = "conv1-1")(input)
    conv1_2 = layers.BatchNormalization(name = "conv1-2")(conv1_1)
    conv1_3 = layers.ReLU(max_value = 6., name = "conv1-3")(conv1_2)

    bottleneck1 = bottleneck(conv1_3, 1, 16, 1, 1)

    bottleneck2 = bottleneck(bottleneck1, 6, 24, 2, 2)
    bottleneck3 = bottleneck(bottleneck2, 6, 24, 1, 3)

    bottleneck4 = bottleneck(bottleneck3, 6, 32, 2, 4)
    bottleneck5 = bottleneck(bottleneck4, 6, 32, 1, 5)
    bottleneck6 = bottleneck(bottleneck5, 6, 32, 1, 6)

    bottleneck7 = bottleneck(bottleneck6, 6, 64, 2, 7)
    bottleneck8 = bottleneck(bottleneck7, 6, 64, 1, 8)
    bottleneck9 = bottleneck(bottleneck8, 6, 64, 1, 9)
    bottleneck10 = bottleneck(bottleneck9, 6, 64, 1, 10)

    bottleneck11 = bottleneck(bottleneck10, 6, 96, 1, 11)
    bottleneck12 = bottleneck(bottleneck11, 6, 96, 1, 12)
    bottleneck13 = bottleneck(bottleneck12, 6, 96, 1, 13)

    bottleneck14 = bottleneck(bottleneck13, 6, 160, 2, 14)
    bottleneck15 = bottleneck(bottleneck14, 6, 160, 1, 15)
    bottleneck16 = bottleneck(bottleneck15, 6, 160, 1, 16)

    bottleneck17 = bottleneck(bottleneck16, 6, 320, 1, 17)

    conv2 = layers.Conv2D(filters = 1280, kernel_size = 1, strides = 1, activation = "relu", name = f"conv2")(bottleneck17)
    global_avg_pool1 = layers.GlobalAveragePooling2D(name = f"globalAvgPool1")(conv2)

    # Couches de classification
    output = layers.Dense(255, activation = "softmax", name = "output")(global_avg_pool1)

    return Model(inputs = input, outputs = output, name = "MobileNetV2")

# 4 - Phase d'entraînement
## Principe d'entrainement

https://www.lebigdata.fr/machine-learning-entrainement-ia


### époque (epoch)

## Présentation des indicateurs de performances

Cette partie a été rédigé sur des travaux antérieurs [HumanForYou - CESI A4](https://github.com/Scordragours/Projet-IA-A4/blob/master/Groupe%20n%C2%B04%20-%20Livrable%20n%C2%B01%20:%20Compte%20rendu%20analyse%20de%20donn%C3%A9es.ipynb).

### Définition

En Machine Learning, la bonne manière de procéder consiste à se baser sur des indicateurs mathématiques précis et concrets ouvrant la porte aux interprétations humaines. Généralement dans ce type de problèmes de classification, des indicateurs tels que des métriques de performance ou encore des courbes de comparaisons sont très utilisés.

Chaque indicateur de cette partie sera donc accompagnée d'une rapide description ainsi qu'une explication sur son utilisation et sur le résultat obtenu dans notre contexte.

!["Visualisation des bonnes et mauvaise données"](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRaefwz7ZSNyFZeO8Urtnjj58MgDKFzRV5aE0WNoXNn6ejBy3ZvVDwQVL8N081CpvTfOJY&usqp=CAU "Visualisation des bonnes et mauvaise données")

### Matrice de confusion

La matrice est de confusion est aujourd'hui l'un des outils les plus utilisés pour évaluer les performances d'un modèle de classification. Celle-ci présente un système de classes associées chacune à une valeur de représentation de cette classe suite à l'entrainement avec un modèle de prédictions. Il est à noter que ce type de matrice fonctionne aussi bien pour des problèmes de classification à 2 classes (binaires) ou plus.

Veuillez trouver ci-dessous un exemple de matrice de prédiction binaire :
!["Matrice de confusion binaire"](https://docs.microsoft.com/fr-fr/dynamics365/finance/finance-insights/media/tn-fn.png "Matrice de confusion binaire")

Comme présenté dans la figure, dans le cas binaire les classes sont :
- ***TP*** : les valeurs prédites positives par le modèle et qui se trouvent être positives au final (bonne prédiction)
- ***TN*** : les valeurs prédites positives par le modèle et qui se trouvent être négatives au final (mauvaise prédiction)
- ***FP*** : les valeurs prédites négatives par le modèle et qui se trouvent être positives au final (mauvaise prédiction)
- ***FN*** : les valeurs prédites négatives par le modèle et qui se trouvent être négatives au final (bonne prédiction)

Ici les expliquation sont pour des cas binaires, mais le fonctionnement est identique.

### Sensibilité, Recall, TPR

La sensibilité mesure la proportion de vrais positifs parmi tous les cas positifs. Le calcule est le suivant : $Recall = {{TP} \over {TP \, + \, FN}}$.

### Spécificité, TNR

La spécificité mesure la proportion de vrais négatifs parmi tous les cas négatifs. Le calcule est le suivant : $TNR = {{TN} \over {TN \, + \, FP}}$.

### Fall out, FPR

Le Fall out mesure la proportion de faux positifs parmi tous les cas négatifs. Le calcul est le suivant : $FPR = {{FP} \over {TN \, + \, FP}}$.

### FNR

Le FNR mesure la proportion de faux négatifs parmi tous les cas positifs. Le calcule est le suivant : $FNR = {{FN} \over {FN \, + \, TP}}$.

### Précision, PPV

La précision mesure la proportion de vrais positifs parmi tous les résultats positifs. Le calcule est le suivant : $Precision = {{TP} \over {TP+FP}}$.

### FDR

Le FDR mesure la proportion de fausses découvertes parmi toutes les découvertes positives. Le calcule est le suivant : $FDR = 1-PPV{{FP} \over {TP \, + \, FP}}$.

### NPV

Len NPV mesure la proportion de vrais négatifs parmi tous les résultats négatifs. Le calcule est le suivant : $NPV = {{TN} \over {TN \, + \, FN}}$.

### FOR

Le FOR mesure la probabilité qu'un résultat négatif prédit soit en réalité un vrai négatif. Le calcule est le suivant : $FOR = 1-NPV{{FN} \over {TN \, + \, FN}}$.

### Accuracy, Acc

L'accurary mesure le rapport entre le nombre de prédictions correctes et le nombre total d'observations. Le calcule est le suivant :  $Accuracy = {{TP \, + \, TN} \over {TP \, + \, TN \, + \, FP \, + \, FN}}$.

### F1-score

Le score F1 représente une évaluation de la performance de l'algorithme. Ce score est la moyenne harmonique de la précision et du rappel (cf: image ci-dessous).

Lorsque deux modèles ont une précision élevée et un faible rappel ou inversement, la comparaison peut être plus compliquée. C'est pourquoi, dans ce type de situation, il est préférable d'utiliser le score F1 car il permet de mesurer ces deux paramètres simultanément.

Le calcule est le suivant : $F1-Score = 2 {{Precision \, * \, Recall} \over {(Precision \, + \, Recall)}}$.

### Courbe ROC

La courbe ROC est un autre moyen d'évaluer un classifieur. Elle confronte le taux de vrai positif (TPR ou recall) au taux de faux positif (FPR).

### AUC

Pour la courbe ROC, un grand taux de vrais positifs implique beaucoup de faux positifs. La diagonale en pointillée représente la courbe ROC d'un classificateur aléatoire. Un classificateur idéal s'en écarte au maximum dans le coin supérieur gauche.

C'est pourquoi on utilise comme métrique de comparaison l'air sous la courbe ROC, nommé AUC, que l'on souhaite la plus proche possible de 1.

## Callback

Les Callback sont des moyens de customiser le comportement d'un modèle durant les phases d'évaluations.

Dans notre cas, nous avons décidé de les utiliser pour :
- Sauvegarder des informations pour Tensorboard.
- Sauvegardes des poids.
- Stopper l'entrainement (Early stopping).
- Création de matrice de confusion.

In [ ]:
import datetime

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

import io
import itertools
import numpy as np

from sklearn.metrics import confusion_matrix, accuracy_score

### Information d'entrainement pour Tensorboard
Le callback [TensorBoard](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) vas enregistrer des informations qui vont permettre à Tenserboard de construire les courbes de progressions de la précision et de la perte.

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = f"logs/fit/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}",
    histogram_freq = 1
)

### Sauvegarde des poids

Le callback [ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) garde en mémoire le poids des modèles. Ils pourront donc être rechargés sur un modèle de même structure à la prochaine exécutions.

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = "logs/weight/cp.ckpt",
    monitor = "val_loss",
    save_weights_only = True,
    save_best_only = True,
    verbose = 1
)

### Stopper l'entrainement (Early stopping)

Le callback [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) va, afin d'éviter le sur-apprentissage, limiter le nombre d'époques. Dès qu'il n'y a pas de progrès pendant un nombre d'époques supérieur à la patience, on se trouve face à un sur-apprentissage et on arrête l'exécution.

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=3,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

### Création de la matrice de confusion

Ici, on construit une matrice de confusion entre chaque époque avec sklearn qui est convertie en png pour la sauvegarder puis l'afficher dans Tensorboard.

Code basé sur le travail de Vincent Havard.

In [ ]:
#fonctions utiles
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

def plot_confusion_matrix(cm_main, class_names, figsize=(5, 5), accuracy_score=None):
    #Crée un matplotlib plot à partir d'une matrice de confusion numérique et des labels associée
    figure = plt.figure(figsize=figsize)
    ax = plt.imshow(cm_main, interpolation='nearest', cmap=plt.cm.Blues)
    str_title = "Confusion matrix"
    if accuracy_score is not None:
        str_title = str_title + f" acc:{np.round(accuracy_score*100, decimals=3)}%"
    plt.title(str_title)

    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)



    threshold = 0.5
    # write labels
    for i, j in itertools.product(range(cm_main.shape[0]), range(cm_main.shape[1])):
        val = cm_main[i, j]
        if val >= 0:
            color = "white" if cm_main[i, j] > threshold else "black"
            plt.text(j, i, cm_main[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def model_to_confusion_matrix(model, x_test, y_test, class_names, plotIt = True):
    y_pred_onehot = model.predict(x_test)
    y_pred = np.argmax(y_pred_onehot, axis=1)
    # Calculate the confusion matrix.
    cm_norm = None
    figure = None
    if y_test.ndim > 1:
        y_test = y_test.squeeze()

    if y_pred.ndim > 1:
        y_pred = y_pred.squeeze()

    idx = np.unique(np.concatenate((y_test, y_pred)))

    cm = confusion_matrix(y_test, y_pred)

    acc = accuracy_score(y_test, y_pred)
    # compute normalzed confusion matrix
    cm_norm = confusion_matrix(y_test, y_pred, normalize="true")
    cm_norm = np.round(cm_norm, decimals=2)
    # print("cm_norm", cm_norm)
    if plotIt:
        cm_main = cm_norm
        # Log the confusion matrix as an image summary.
        n=len(class_names)
        figure = plot_confusion_matrix(cm_main, class_names, figsize=(n,n), accuracy_score=acc)
    # cm, y_pred, y_pred_onehot, (optional: cm_norm, figure)
    return cm, y_pred, y_pred_onehot, cm_norm, figure

In [ ]:
#définition du callback perso
class ConfusionMatrixCallback(keras.callbacks.Callback):
    def __init__(self, x_test, y_test, class_names, file_writer_cm):

        self.x_test = x_test
        self.y_test = y_test
        self.class_names = class_names
        self.file_writer_cm = file_writer_cm

    def on_epoch_end(self, epoch, logs=None):
        _, _, _, _, figure = model_to_confusion_matrix(self.model, self.x_test,self.y_test,self.class_names)
        # Log the confusion matrix as an image summary.
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with self.file_writer_cm.as_default():

            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            self.file_writer_cm.flush()
            print(self.file_writer_cm)

# 5 - Exécution des Modèles
## Automatisation

In [ ]:
from csv import DictWriter

In [ ]:
def accuracy_save(path_models, n_dropout, nb_epoch, min_delta, patience, accuracy ):
    # Import DictWriter class from CSV module

    # list of column names
    field_names = ['N_Dropout','Nb_Epoch', 'Min_Delta', 'Patience', 'Accuracy']

    with open(f"{path_models}/Graphe.csv", 'a') as f_object:
        writer_object = DictWriter(f_object, fieldnames=field_names)

        writer_object.writerow({
            'N_Dropout':n_dropout,
            'Nb_Epoch': nb_epoch,
            'Min_Delta':min_delta,
            'Patience':patience,
            'Accuracy': accuracy
        })

        f_object.close()

In [ ]:
def auto_models(name, model, dataset, config):
    path_models = f"../models/{name}"

    class_names =  dataset["train"].class_names
    x_test = []
    y_test = []
    for images, labels in dataset["validation"].take(-1):
        x_test = images.numpy()
        y_test = labels.numpy()

    model.compile(
        optimizer = "adam",
        loss = losses.SparseCategoricalCrossentropy(from_logits = True),
        metrics = [ "accuracy" ]
    )

    if not os.path.exists(path_models):
        os.mkdir(path_models)

        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir = f"{path_models}/tensorBoard/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
        )

        cp_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath = f"{path_models}/weight/cp.ckpt",
            monitor = "val_loss",
            save_weights_only = config["save_weights_only"],
            save_best_only = config["save_best_only"]
        )

        early_stopping_callback = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta = config["min_delta"],
            patience = config["patience"]
        )

        confusionMatrix_callback = ConfusionMatrixCallback(
            x_test = x_test,
            y_test = y_test,
            class_names = class_names,
            file_writer_cm = tf.summary.create_file_writer(f"{path_models}/confusionMatrix")
        )

        history = model.fit(
            dataset["train"],
            validation_data = dataset["validation"],
            callbacks = [
                tensorboard_callback,
                cp_callback,
                confusionMatrix_callback,
                early_stopping_callback
            ],
            batch_size = 1,
            epochs = config["epochs"]
        )

        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']

        print(acc, val_acc)

    test_loss, test_acc = model.evaluate(dataset["test"], verbose = 2)
    print(test_loss, test_acc)

    accuracy_save(
        path_models,
        config["dropout"],
        config["epochs"],
        config["min_delta"],
        config["patience"],
        test_acc
    )

## Exécution sans data augmentation

In [ ]:
train_set, validation_set, test_set = loadDataSet(path_datasets, (256, 256))

config_colored = {
    "train": train_set,
    "validation": validation_set,
    "test": test_set
}

### Perceptron

In [ ]:
auto_models(
    name = "perceptron-colored",
    model = perceptron((256, 256, 3), 0.5),
    dataset = config_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### AlexNet

In [ ]:
auto_models(
    name = "alexNet-colored",
    model = alex_net((256, 256, 3)),
    dataset = config_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### VGG 16

In [ ]:
auto_models(
    name = "vgg16-colored",
    model = vgg_16((512, 512, 3)),
    dataset = config_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### VGG 19

In [ ]:
auto_models(
    name = "vgg19-colored",
    model = vgg_19((512, 512, 3)),
    dataset = config_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### ResNet 34 - Plain

In [ ]:
auto_models(
    name = "resNet34-plain-colored",
    model = resnet_34_layers((512, 512, 3), False),
    dataset = config_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### ResNet 34 - Residual

In [ ]:
auto_models(
    name = "resNet34-residual-colored",
    model = resnet_34_layers((512, 512, 3), True),
    dataset = config_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### ResNet 50 - Plain

In [ ]:
auto_models(
    name = "resNet50-plain-colored",
    model = resnet_50_layers((512, 512, 3), False),
    dataset = config_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### ResNet 50 - Residual

In [ ]:
auto_models(
    name = "resNet50-residual-colored",
    model = resnet_50_layers((512, 512, 3), True),
    dataset = config_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### MobileNet V1

In [ ]:
auto_models(
    name = "mobileNetV1-colored",
    model = mobilenet_v1((512, 512, 3)),
    dataset = config_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### MobileNet V2

In [ ]:
auto_models(
    name = "mobileNetV2-colored",
    model = mobilenet_v2((512, 512, 3)),
    dataset = config_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

## Exécution avec data augmentation

In [ ]:
train_augmented_colored_set, validation_augmented_colored_set, test_augmented_colored_set = loadDataSet(path_datasets_augmented_trainTest, (512, 512))

config_augmented_colored = {
    "train": train_augmented_colored_set,
    "validation": validation_augmented_colored_set,
    "test": test_augmented_colored_set
}

### Perceptron

In [ ]:
auto_models(
    name = "perceptron-augmented-colored",
    model = perceptron((512, 512, 3), 0.5),
    dataset = config_augmented_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### AlexNet

In [ ]:
auto_models(
    name = "alexNet-augmented-colored",
    model = alex_net((512, 512, 3)),
    dataset = config_augmented_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### VGG 16

In [ ]:
auto_models(
    name = "vgg16-augmented-colored",
    model = vgg_16((512, 512, 3)),
    dataset = config_augmented_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### VGG 19

In [ ]:
auto_models(
    name = "vgg19-augmented-colored",
    model = vgg_19((512, 512, 3)),
    dataset = config_augmented_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### ResNet 34 - Plain

In [ ]:
auto_models(
    name = "resNet34-augmented-plain-colored",
    model = resnet_34_layers((512, 512, 3), False),
    dataset = config_augmented_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### ResNet 34 - Residual

In [ ]:
auto_models(
    name = "resNet34-augmented-residual-colored",
    model = resnet_34_layers((512, 512, 3), True),
    dataset = config_augmented_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### ResNet 50 - Plain

In [ ]:
auto_models(
    name = "resNet50-augmented-plain-colored",
    model = resnet_50_layers((512, 512, 3), False),
    dataset = config_augmented_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### ResNet 50 - Residual

In [ ]:
auto_models(
    name = "resNet50-augmented-residual-colored",
    model = resnet_50_layers((512, 512, 3), True),
    dataset = config_augmented_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### MobileNet V1

In [ ]:
auto_models(
    name = "mobileNetV1-augmented-colored",
    model = mobilenet_v1((512, 512, 3)),
    dataset = config_augmented_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)

### MobileNet V2

In [ ]:
auto_models(
    name = "mobileNetV2-augmented-colored",
    model = mobilenet_v2((512, 512, 3)),
    dataset = config_augmented_colored,
    config = {
        "save_weights_only": True,
        "save_best_only": True,
        "min_delta": 0,
        "patience": 5,
        "epochs": 20,
        "dropout": 0.3
    }
)